# กลไก Attention และ Transformer

ข้อเสียสำคัญของเครือข่ายแบบ recurrent คือทุกคำในลำดับมีผลกระทบต่อผลลัพธ์เท่ากัน ซึ่งทำให้ประสิทธิภาพของโมเดล LSTM encoder-decoder มาตรฐานสำหรับงานลำดับต่อลำดับ เช่น การระบุชื่อเฉพาะ (Named Entity Recognition) และการแปลภาษา ไม่ดีเท่าที่ควร ในความเป็นจริง คำบางคำในลำดับข้อมูลนำเข้ามักมีผลกระทบต่อผลลัพธ์มากกว่าคำอื่น ๆ

ลองพิจารณาโมเดลลำดับต่อลำดับ เช่น การแปลภาษา โมเดลนี้ถูกสร้างขึ้นโดยใช้เครือข่าย recurrent สองตัว โดยเครือข่ายตัวหนึ่ง (**encoder**) จะบีบอัดลำดับข้อมูลนำเข้าให้เป็น hidden state และอีกตัวหนึ่ง (**decoder**) จะคลาย hidden state นี้ออกมาเป็นผลลัพธ์ที่แปลแล้ว ปัญหาของวิธีนี้คือ hidden state สุดท้ายของเครือข่ายจะจำจุดเริ่มต้นของประโยคได้ยาก ส่งผลให้คุณภาพของโมเดลลดลงเมื่อประโยคยาวขึ้น

**กลไก Attention** เป็นวิธีการที่ช่วยให้น้ำหนักของผลกระทบเชิงบริบทของแต่ละเวกเตอร์นำเข้าต่อการทำนายผลลัพธ์ของ RNN แตกต่างกัน วิธีการนี้ถูกนำมาใช้โดยการสร้างทางลัดระหว่างสถานะกลางของ RNN นำเข้าและ RNN ผลลัพธ์ ด้วยวิธีนี้ เมื่อสร้างสัญลักษณ์ผลลัพธ์ $y_t$ เราจะพิจารณา hidden states นำเข้าทั้งหมด $h_i$ โดยมีค่าสัมประสิทธิ์น้ำหนักที่แตกต่างกัน $\alpha_{t,i}$

![ภาพแสดงโมเดล encoder/decoder พร้อมชั้น attention แบบ additive](../../../../../translated_images/th/encoder-decoder-attention.7a726296894fb567.webp)
*โมเดล encoder-decoder พร้อมกลไก attention แบบ additive จาก [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) อ้างอิงจาก [บล็อกโพสต์นี้](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

เมทริกซ์ Attention $\{\alpha_{i,j}\}$ แสดงระดับที่คำบางคำในข้อมูลนำเข้ามีบทบาทในการสร้างคำในลำดับผลลัพธ์ ตัวอย่างของเมทริกซ์ดังกล่าวแสดงอยู่ด้านล่าง:

![ภาพแสดงตัวอย่างการจัดแนวที่พบโดย RNNsearch-50 จาก Bahdanau - arviz.org](../../../../../translated_images/th/bahdanau-fig3.09ba2d37f202a6af.webp)

*ภาพจาก [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

กลไก Attention มีบทบาทสำคัญในความก้าวหน้าของงานประมวลผลภาษาธรรมชาติในปัจจุบันหรือใกล้เคียงปัจจุบัน อย่างไรก็ตาม การเพิ่ม Attention ทำให้จำนวนพารามิเตอร์ของโมเดลเพิ่มขึ้นอย่างมาก ซึ่งนำไปสู่ปัญหาการปรับขนาดใน RNNs ข้อจำกัดสำคัญของการปรับขนาด RNNs คือธรรมชาติของโมเดลที่ต้องประมวลผลลำดับทีละขั้นตอน ซึ่งทำให้การ batch และการประมวลผลแบบขนานทำได้ยาก ใน RNN แต่ละองค์ประกอบของลำดับต้องถูกประมวลผลตามลำดับ ซึ่งหมายความว่าไม่สามารถประมวลผลแบบขนานได้ง่าย

การนำกลไก Attention มาใช้ร่วมกับข้อจำกัดนี้นำไปสู่การสร้างโมเดล Transformer ซึ่งเป็น State of the Art ในปัจจุบันที่เราใช้กัน เช่น BERT และ OpenGPT3

## โมเดล Transformer

แทนที่จะส่งต่อบริบทของการทำนายครั้งก่อนเข้าสู่ขั้นตอนการประเมินครั้งถัดไป **โมเดล Transformer** ใช้ **positional encodings** และ Attention เพื่อจับบริบทของข้อมูลนำเข้าภายในหน้าต่างข้อความที่กำหนด ภาพด้านล่างแสดงวิธีที่ positional encodings ร่วมกับ Attention สามารถจับบริบทภายในหน้าต่างที่กำหนดได้

![GIF แสดงวิธีการประเมินในโมเดล Transformer](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

เนื่องจากแต่ละตำแหน่งนำเข้าถูกแมปไปยังตำแหน่งผลลัพธ์อย่างอิสระ Transformers สามารถประมวลผลแบบขนานได้ดีกว่า RNNs ซึ่งช่วยให้สร้างโมเดลภาษาที่ใหญ่ขึ้นและแสดงออกได้มากขึ้น หัว Attention แต่ละหัวสามารถใช้เรียนรู้ความสัมพันธ์ระหว่างคำที่แตกต่างกัน ซึ่งช่วยปรับปรุงงานประมวลผลภาษาธรรมชาติในขั้นตอนต่อไป

**BERT** (Bidirectional Encoder Representations from Transformers) เป็นเครือข่าย Transformer ขนาดใหญ่มากที่มีหลายชั้น โดย *BERT-base* มี 12 ชั้น และ *BERT-large* มี 24 ชั้น โมเดลนี้ถูก pre-trained บนชุดข้อมูลข้อความขนาดใหญ่ (WikiPedia + หนังสือ) โดยใช้การฝึกแบบ unsupervised (การทำนายคำที่ถูก mask ในประโยค) ในระหว่างการ pre-training โมเดลจะเรียนรู้ความเข้าใจภาษาระดับสูง ซึ่งสามารถนำไปใช้กับชุดข้อมูลอื่น ๆ ผ่านการ fine tuning กระบวนการนี้เรียกว่า **transfer learning**

![ภาพจาก http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/th/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

มีสถาปัตยกรรม Transformer หลากหลายรูปแบบ เช่น BERT, DistilBERT, BigBird, OpenGPT3 และอื่น ๆ ที่สามารถนำไป fine tune ได้ [HuggingFace package](https://github.com/huggingface/) มี repository สำหรับการฝึกสถาปัตยกรรมเหล่านี้หลายตัวด้วย PyTorch

## การใช้ BERT สำหรับการจัดประเภทข้อความ

มาดูกันว่าเราสามารถใช้โมเดล BERT ที่ถูก pre-trained เพื่อแก้ปัญหางานดั้งเดิมของเรา: การจัดประเภทลำดับข้อความ เราจะจัดประเภทชุดข้อมูล AG News ดั้งเดิมของเรา

เริ่มต้นด้วยการโหลดไลบรารี HuggingFace และชุดข้อมูลของเรา:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


เนื่องจากเราจะใช้โมเดล BERT ที่ผ่านการฝึกมาแล้ว เราจำเป็นต้องใช้ตัวตัดคำเฉพาะสำหรับโมเดลนี้ ก่อนอื่น เราจะโหลดตัวตัดคำที่เชื่อมโยงกับโมเดล BERT ที่ผ่านการฝึกมาแล้ว

ไลบรารี HuggingFace มีคลังเก็บโมเดลที่ผ่านการฝึกมาแล้ว ซึ่งคุณสามารถใช้งานได้เพียงแค่ระบุชื่อโมเดลเป็นอาร์กิวเมนต์ในฟังก์ชัน `from_pretrained` ไฟล์ไบนารีที่จำเป็นทั้งหมดสำหรับโมเดลจะถูกดาวน์โหลดโดยอัตโนมัติ

อย่างไรก็ตาม ในบางกรณีคุณอาจต้องโหลดโมเดลของคุณเอง ซึ่งในกรณีนี้คุณสามารถระบุไดเรกทอรีที่มีไฟล์ที่เกี่ยวข้องทั้งหมดได้ เช่น พารามิเตอร์สำหรับตัวตัดคำ ไฟล์ `config.json` ที่มีพารามิเตอร์ของโมเดล น้ำหนักไบนารี เป็นต้น


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

วัตถุ `tokenizer` มีฟังก์ชัน `encode` ที่สามารถใช้เข้ารหัสข้อความได้โดยตรง:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

จากนั้น มาสร้างตัววนซ้ำที่เราจะใช้ระหว่างการฝึกอบรมเพื่อเข้าถึงข้อมูล เนื่องจาก BERT ใช้ฟังก์ชันการเข้ารหัสของตัวเอง เราจำเป็นต้องกำหนดฟังก์ชันการเติมข้อมูลที่คล้ายกับ `padify` ที่เราได้กำหนดไว้ก่อนหน้านี้:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

ในกรณีของเรา เราจะใช้โมเดล BERT ที่ผ่านการฝึกมาแล้วชื่อว่า `bert-base-uncased` มาลองโหลดโมเดลโดยใช้แพ็กเกจ `BertForSequenceClassfication` วิธีนี้จะช่วยให้โมเดลของเรามีสถาปัตยกรรมที่จำเป็นสำหรับการจัดประเภท รวมถึงตัวจัดประเภทสุดท้าย คุณจะเห็นข้อความเตือนที่ระบุว่าน้ำหนักของตัวจัดประเภทสุดท้ายยังไม่ได้รับการกำหนดค่า และโมเดลจะต้องการการฝึกเพิ่มเติม - ซึ่งไม่มีปัญหาเลย เพราะนั่นคือสิ่งที่เรากำลังจะทำ!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

ตอนนี้เราเริ่มการฝึกอบรมได้เลย! เนื่องจาก BERT ได้รับการฝึกอบรมล่วงหน้าแล้ว เราจึงต้องการเริ่มต้นด้วยอัตราการเรียนรู้ที่ค่อนข้างต่ำเพื่อไม่ให้ทำลายน้ำหนักเริ่มต้น

งานหนักทั้งหมดดำเนินการโดยโมเดล `BertForSequenceClassification` เมื่อเราเรียกใช้โมเดลกับข้อมูลการฝึกอบรม โมเดลจะคืนค่าทั้ง loss และผลลัพธ์ของเครือข่ายสำหรับ minibatch ที่ป้อนเข้า เราใช้ loss สำหรับการปรับพารามิเตอร์ (`loss.backward()` ทำการถอยหลัง) และใช้ `out` สำหรับคำนวณความแม่นยำของการฝึกอบรมโดยการเปรียบเทียบป้ายกำกับที่ได้ `labs` (คำนวณโดยใช้ `argmax`) กับป้ายกำกับที่คาดหวัง `labels`

เพื่อควบคุมกระบวนการ เราสะสมค่า loss และความแม่นยำในหลายๆ รอบ และพิมพ์ผลลัพธ์ทุกๆ รอบการฝึกอบรมตามค่า `report_freq`

การฝึกอบรมนี้อาจใช้เวลานานพอสมควร ดังนั้นเราจึงจำกัดจำนวนรอบการฝึกอบรม


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


คุณสามารถเห็นได้ (โดยเฉพาะถ้าคุณเพิ่มจำนวนรอบการทำงานและรอให้นานพอ) ว่าการจัดประเภทด้วย BERT ให้ความแม่นยำที่ดีมาก! นั่นเป็นเพราะว่า BERT เข้าใจโครงสร้างของภาษาได้ดีอยู่แล้ว และเราจำเป็นต้องปรับแต่งตัวจัดประเภทขั้นสุดท้ายเท่านั้น อย่างไรก็ตาม เนื่องจาก BERT เป็นโมเดลขนาดใหญ่ กระบวนการฝึกทั้งหมดจึงใช้เวลานาน และต้องการพลังการประมวลผลที่สูงมาก! (GPU และควรมีมากกว่าหนึ่งตัว)

> **Note:** ในตัวอย่างของเรา เราได้ใช้หนึ่งในโมเดล BERT ที่ผ่านการฝึกฝนล่วงหน้าขนาดเล็กที่สุด ยังมีโมเดลที่ใหญ่กว่าซึ่งมีแนวโน้มที่จะให้ผลลัพธ์ที่ดีกว่า


## การประเมินประสิทธิภาพของโมเดล

ตอนนี้เราสามารถประเมินประสิทธิภาพของโมเดลบนชุดข้อมูลทดสอบได้แล้ว วงจรการประเมินผลมีความคล้ายคลึงกับวงจรการฝึกอบรม แต่เราต้องไม่ลืมเปลี่ยนโมเดลไปเป็นโหมดการประเมินผลโดยการเรียกใช้ `model.eval()`


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## ข้อคิดสำคัญ

ในบทนี้ เราได้เห็นแล้วว่าการนำโมเดลภาษาที่ถูกฝึกมาแล้วจากไลบรารี **transformers** มาปรับใช้กับงานการจัดประเภทข้อความนั้นง่ายเพียงใด นอกจากนี้ โมเดล BERT ยังสามารถนำไปใช้กับงานดึงข้อมูลเอนทิตี การตอบคำถาม และงาน NLP อื่น ๆ ได้อีกด้วย

โมเดล Transformer ถือเป็นเทคโนโลยีที่ล้ำหน้าที่สุดในปัจจุบันสำหรับงาน NLP และในหลายกรณีควรเป็นทางเลือกแรกที่คุณเริ่มทดลองใช้งานเมื่อพัฒนาวิธีแก้ปัญหา NLP แบบเฉพาะ อย่างไรก็ตาม การทำความเข้าใจหลักการพื้นฐานของเครือข่ายประสาทแบบวนซ้ำที่ได้กล่าวถึงในโมดูลนี้มีความสำคัญอย่างยิ่ง หากคุณต้องการสร้างโมเดลประสาทขั้นสูง



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้อง แต่โปรดทราบว่าการแปลโดยอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาดั้งเดิมควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษามืออาชีพ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความผิดที่เกิดจากการใช้การแปลนี้
